In [1]:
import pandas as pd

In [ ]:

one = pd.read_csv('one_rows.csv', header=None)
zero = pd.read_csv('zero_rows.csv', header=None)

print(one.shape)
print(zero.shape)
# 把zero随机删除951行
zero = zero.drop(zero.sample(n=951).index)
zero.reset_index(drop=True, inplace=True)
print(zero.shape)
# save csv
zero.to_csv('zero_rows_del.csv', index=False, header=False)
print('zero')
# 从zero随机取2150行
zero_t = zero.sample(n=2150)
# 获取这些抽取行的索引
zero_t_indices = zero_t.index
# 删除这些行
zero_v = zero.drop(zero_t_indices)
print('one')
# 从one随机取430行
one_t = one.sample(n=2150)
# 获取这些抽取行的索引
one_t_indices = one_t.index
# 删除这些行
one_v = one.drop(one_t_indices)
print('combine_t')
# 交叉合并 zero_t 和 one_t
# 创建一个新的 DataFrame，用于交叉合并 zero_t 和 one_t
combined_t = pd.DataFrame()

# 交替添加 zero_t 和 one_t 的行
for i in range(2150):
    # print(i)
    combined_t = pd.concat([combined_t, zero_t.iloc[[i]]])
    combined_t = pd.concat([combined_t, one_t.iloc[[i]]])

# 重置索引
combined_t = combined_t.reset_index(drop=True)
combined_t.to_csv('train.csv', index=False, header=False)
print(combined_t.shape)

print('combine_v')
combined_v = pd.DataFrame()

# 交替添加 zero_t 和 one_t 的行
for i in range(430):
    combined_v = pd.concat([combined_v, zero_v.iloc[[i]]])
    combined_v = pd.concat([combined_v, one_v.iloc[[i]]])

# 重置索引
combined_v = combined_v.reset_index(drop=True)
combined_v.to_csv('valid.csv', index=False, header=False)
print(combined_v.shape)


In [10]:
import pandas as pd
one = pd.read_csv('/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/Gm/sorted_data.csv', header=None)
# zero = pd.read_csv('zero_rows.csv', header=None)

print(one[1].apply(str).apply(len).describe())
# print(zero[1].apply(str).apply(len).describe())


count     7198.000000
mean      4610.921506
std       4112.498261
min          8.000000
25%       2131.250000
50%       3699.000000
75%       5736.750000
max      56556.000000
Name: 1, dtype: float64


In [3]:
# 对数据从小到大排序

import pandas as pd

# 读取CSV文件
csv_file_path = '/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/Os/data.csv'  # 替换为你的CSV文件路径
data = pd.read_csv(csv_file_path, header=None)

# 添加一列，计算第二列基因序列的长度
data['Sequence_Length'] = data.iloc[:, 1].apply(len)

# 按照序列长度排序
sorted_data = data.sort_values(by='Sequence_Length')

# 删除临时的长度列（可选）
sorted_data = sorted_data.drop(columns=['Sequence_Length'])

# 保存排序后的数据到一个新的CSV文件
sorted_csv_file_path = '/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/Os/sorted_data.csv'  # 替换为你想要保存的文件路径
sorted_data.to_csv(sorted_csv_file_path, index=False, header=False)

print(f"排序后的数据已保存到 {sorted_csv_file_path}")


排序后的数据已保存到 /Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/Os/sorted_data.csv


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np

# 独热编码函数
def one_hot_encode(sequence, max_len, vocab_size):
    encoding_map = {'A': 0, 'C': 1, 'G': 2, 'T': 3, 'N': 4}
    one_hot_encoded = np.zeros((max_len, vocab_size), dtype=int)

    for i, base in enumerate(sequence):
        if i >= max_len:
            break
        index = encoding_map.get(base, -1)
        if index != -1:
            one_hot_encoded[i, index] = 1

    return one_hot_encoded

# 自定义数据集类
class GeneDataset(Dataset):
    def __init__(self, file_path, max_len, vocab_size):
        self.data = pd.read_csv(file_path, header=None, skiprows=0)
        print(self.data[2])
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.data['encoded_sequence'] = self.data[1].apply(lambda seq: one_hot_encode(seq, max_len, vocab_size)) # type: ignore
        self.data['length'] = self.data[1].apply(len)
        self.labels = torch.tensor(self.data[2].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        
        encoded_sequence = self.data.iloc[idx]['encoded_sequence']
        length = self.data.iloc[idx]['length']
        label = self.labels[idx]
        return torch.tensor(encoded_sequence, dtype=torch.float32), torch.tensor(length), label

# 生成数据加载器的批处理函数
def collate_fn(batch):
    sequences, lengths, labels = zip(*batch)

    # 转换为tensor
    sequences = [torch.tensor(seq) for seq in sequences]
    lengths = torch.tensor(lengths, dtype=torch.long)
    labels = torch.tensor(labels, dtype=torch.long)

    # 填充序列
    padded_sequences = pad_sequence(sequences, batch_first=True)

    # 打包序列
    packed_sequences = pack_padded_sequence(padded_sequences, lengths, batch_first=True, enforce_sorted=False)
    return packed_sequences, lengths, labels


# 测试数据加载器和GRU模型
def main():
    # 参数设置
    file_path = '/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/At/sorted_data.csv'  
    max_len = 23544  # 最大序列长度
    vocab_size = 5  # 独热编码的维度 (A, C, G, T, N)
 
    batch_size = 32  # 批次大小


    # 创建数据集和数据加载器
    dataset = GeneDataset(file_path, max_len, vocab_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    
    # 把dataloaders里的每个批次的数据存起来
    import pickle
    for i, batch in enumerate(dataloader):
        
        with open(f'/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/二进制GRU/排序好/At/data_batch_{i}.pkl', 'wb') as f:
            pickle.dump(batch, f)



if __name__ == "__main__":
    main()


0       label
1           0
2           1
3           1
4           1
        ...  
6475        0
6476        0
6477        0
6478        0
6479        1
Name: 2, Length: 6480, dtype: object


KeyboardInterrupt: 

NameError: name 'dataset' is not defined

In [9]:
# 读取/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/二进制GRU/排序好/At/data_batch_0.pkl
import pickle
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
with open('/Data4/gly_wkdir/coldgenepredict/raw_sec/S_italica/分好的数据集csv/二进制GRU/排序好/At/data_batch_202.pkl', 'rb') as f:
    data = pickle.load(f)
    print(data[0])
    print(data[1])
    print(data[2])

PackedSequence(data=tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]]), batch_sizes=tensor([15, 15, 15,  ...,  1,  1,  1]), sorted_indices=tensor([ 8,  6,  2,  4, 14, 10, 12,  7,  3, 11,  1,  0, 13,  9,  5]), unsorted_indices=tensor([11, 10,  2,  8,  3, 14,  1,  7,  0, 13,  5,  9,  6, 12,  4]))
tensor([ 1775,  1892,  4858,  2624,  3849,  1010,  4965,  2845, 10519,  1359,
         2912,  2494,  2869,  1670,  3744])
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0])


PackedSequence(data=tensor([[0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]]), batch_sizes=tensor([15, 15, 15,  ...,  1,  1,  1]), sorted_indices=tensor([ 8,  6,  2,  4, 14, 10, 12,  7,  3, 11,  1,  0, 13,  9,  5]), unsorted_indices=tensor([11, 10,  2,  8,  3, 14,  1,  7,  0, 13,  5,  9,  6, 12,  4]))
tensor([ 1775,  1892,  4858,  2624,  3849,  1010,  4965,  2845, 10519,  1359,
         2912,  2494,  2869,  1670,  3744])
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0])

In [ ]:
1